In [1]:
# import necessary libraries
import requests 
import pandas as pd 
import numpy as np 
import random 
from pandas.io.json import json_normalize
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim
!conda install -c conda-forge folium=0.5.0 --yes
import folium 

print("Finished importing everything!")

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Finished importing everything!


In [2]:
# set up Foursquare interaction
CLIENT_ID = 'IUCBY4TBYNGIPFI0UYJP15RSIWGXC2NW2JF11HC51OA4NL2G' 
CLIENT_SECRET = 'IXR3OFKBD0GKPCFHBDXTYDZWVAFYD1RSXKVD5OUVNHDYXVTN' 
VERSION = '20180604'
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: IUCBY4TBYNGIPFI0UYJP15RSIWGXC2NW2JF11HC51OA4NL2G
CLIENT_SECRET:IXR3OFKBD0GKPCFHBDXTYDZWVAFYD1RSXKVD5OUVNHDYXVTN


In [3]:
# set up Foursquare location data based around 30th Street Station in Philadelphia
address = '2955 Market St, Philadelphia, PA'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

39.955935 -75.1818696


In [4]:
# define initial URL using Arcade as query parameter and create url without limit since data is hardcapped at 50 anyways
radius = 12000
search_query = "Arcade"
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius)
url

'https://api.foursquare.com/v2/venues/search?client_id=IUCBY4TBYNGIPFI0UYJP15RSIWGXC2NW2JF11HC51OA4NL2G&client_secret=IXR3OFKBD0GKPCFHBDXTYDZWVAFYD1RSXKVD5OUVNHDYXVTN&ll=39.955935,-75.1818696&v=20180604&query=Arcade&radius=12000'

In [5]:
# pull json results
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ed001aa9388d7001b61ee95'},
 'response': {'venues': [{'id': '4ed0447930f81894ae34d890',
    'name': "Pappas' Arcade",
    'location': {'lat': 39.941586,
     'lng': -75.172937,
     'labeledLatLngs': [{'label': 'display',
       'lat': 39.941586,
       'lng': -75.172937}],
     'distance': 1769,
     'postalCode': '19146',
     'cc': 'US',
     'city': 'Philadelphia',
     'state': 'PA',
     'country': 'United States',
     'formattedAddress': ['Philadelphia, PA 19146', 'United States']},
    'categories': [{'id': '4bf58dd8d48988d1d4941735',
      'name': 'Speakeasy',
      'pluralName': 'Speakeasies',
      'shortName': 'Speakeasy',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/secretbar_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1590690379',
    'hasPerk': False},
   {'id': '51170d2fe4b06e75e1504bb4',
    'name': '40th St Arcade',
    'location': {'lat': 39.9360768845371,
     'lng': -

In [6]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.shape

C:\Users\dsalt\anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


(7, 18)

In [7]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# perform extraction and rename relevant columns
dataframe['categories'] = dataframe.apply(get_category_type, axis=1)
dataframe.rename(columns={"location.lat": "lat", "location.lng": "lng"}, inplace=True)
dataframe.head(50)

,id,name,categories,referralId,hasPerk,lat,lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.address,location.crossStreet,venuePage.id
0,4ed0447930f81894ae34d890,Pappas' Arcade,Speakeasy,v-1590690379,False,39.941586,-75.172937,"[{'label': 'display', 'lat': 39.941586, 'lng':...",1769,19146,US,Philadelphia,PA,United States,"[Philadelphia, PA 19146, United States]",NaN,NaN,NaN
1,51170d2fe4b06e75e1504bb4,40th St Arcade,Arcade,v-1590690379,False,39.936077,-75.200354,"[{'label': 'display', 'lat': 39.9360768845371,...",2715,19104,US,Philadelphia,PA,United States,"[Philadelphia, PA 19104, United States]",NaN,NaN,NaN
2,53d4475b498ea2e8032fc213,Keystone Mini-Golf and Arcade,Mini Golf,v-1590690379,False,39.976278,-75.137359,"[{'label': 'display', 'lat': 39.97627814228616...",4421,19122,US,Philadelphia,PA,United States,"[161 Cecil B Moore Ave (North Palethorp St), P...",161 Cecil B Moore Ave,North Palethorp St,93750609
3,4e05f8881838eb15aef77b2e,Claudys Arcade,Arts & Entertainment,v-1590690379,False,39.956537,-75.070635,"[{'label': 'display', 'lat': 39.956537, 'lng':...",9491,NaN,US,Pennsauken,NJ,United States,"[Me, Myself & I, Pennsauken, NJ, United States]","Me, Myself & I",NaN,NaN
4,4e05f8c6b0fbba5eee1269da,Claudy's Xbox arcade,Arcade,v-1590690379,False,39.956501,-75.070283,"[{'label': 'display', 'lat': 39.95650088914203...",9521,NaN,US,Pennsauken,NJ,United States,"[Home on couch, Pennsauken, NJ, United States]",Home on couch,NaN,NaN
5,40b28c80f964a520d3ff1ee3,Old Mill Restaurant & Arcade,American Restaurant,v-1590690379,False,40.041907,-75.136629,"[{'label': 'display', 'lat': 40.04190664195742...",10318,08062,US,Mullica Hill,PA,United States,"[1 S Main St (Mill Rd), Mullica Hill, PA 08062...",1 S Main St,Mill Rd,NaN
6,4d3b6d0ebf6d5481c8fbd6e1,La Iglesia Arca de Salvación,Church,v-1590690379,False,39.948833,-75.113890,"[{'label': 'display', 'lat': 39.948833, 'lng':...",5854,NaN,US,NaN,New Jersey,United States,"[New Jersey, United States]",NaN,NaN,NaN


In [8]:
# Drop bank (coin counter) and church (Spanish church)
dataframe = dataframe[dataframe.categories != 'Bank']
dataframe = dataframe[dataframe.categories != 'Church']
dataframe.head(20)

,id,name,categories,referralId,hasPerk,lat,lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.address,location.crossStreet,venuePage.id
0,4ed0447930f81894ae34d890,Pappas' Arcade,Speakeasy,v-1590690379,False,39.941586,-75.172937,"[{'label': 'display', 'lat': 39.941586, 'lng':...",1769,19146,US,Philadelphia,PA,United States,"[Philadelphia, PA 19146, United States]",NaN,NaN,NaN
1,51170d2fe4b06e75e1504bb4,40th St Arcade,Arcade,v-1590690379,False,39.936077,-75.200354,"[{'label': 'display', 'lat': 39.9360768845371,...",2715,19104,US,Philadelphia,PA,United States,"[Philadelphia, PA 19104, United States]",NaN,NaN,NaN
2,53d4475b498ea2e8032fc213,Keystone Mini-Golf and Arcade,Mini Golf,v-1590690379,False,39.976278,-75.137359,"[{'label': 'display', 'lat': 39.97627814228616...",4421,19122,US,Philadelphia,PA,United States,"[161 Cecil B Moore Ave (North Palethorp St), P...",161 Cecil B Moore Ave,North Palethorp St,93750609
3,4e05f8881838eb15aef77b2e,Claudys Arcade,Arts & Entertainment,v-1590690379,False,39.956537,-75.070635,"[{'label': 'display', 'lat': 39.956537, 'lng':...",9491,NaN,US,Pennsauken,NJ,United States,"[Me, Myself & I, Pennsauken, NJ, United States]","Me, Myself & I",NaN,NaN
4,4e05f8c6b0fbba5eee1269da,Claudy's Xbox arcade,Arcade,v-1590690379,False,39.956501,-75.070283,"[{'label': 'display', 'lat': 39.95650088914203...",9521,NaN,US,Pennsauken,NJ,United States,"[Home on couch, Pennsauken, NJ, United States]",Home on couch,NaN,NaN
5,40b28c80f964a520d3ff1ee3,Old Mill Restaurant & Arcade,American Restaurant,v-1590690379,False,40.041907,-75.136629,"[{'label': 'display', 'lat': 40.04190664195742...",10318,08062,US,Mullica Hill,PA,United States,"[1 S Main St (Mill Rd), Mullica Hill, PA 08062...",1 S Main St,Mill Rd,NaN


In [9]:
# define second query for Game related venues, and run through same process
search_query2 = "Game" 
url2 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query2, radius)
results_game = requests.get(url2).json()

# assign relevant part of JSON to venues
venues_game = results_game['response']['venues']

# tranform venues into a dataframe
dataframe_game = json_normalize(venues_game)
dataframe_game.shape

C:\Users\dsalt\anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  # Remove the CWD from sys.path while we load stuff.


(30, 25)

In [10]:
# perform category extraction
dataframe_game['categories'] = dataframe_game.apply(get_category_type, axis=1)
dataframe_game.rename(columns={"location.lat": "lat", "location.lng": "lng"}, inplace=True)
dataframe_game.head(50)

,id,name,categories,referralId,hasPerk,lat,lng,location.labeledLatLngs,location.distance,location.postalCode,...,location.address,location.crossStreet,location.neighborhood,delivery.id,delivery.url,delivery.provider.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.icon.name,venuePage.id
0,5b0b0889588e36002cb07e23,Game Masters,Exhibit,v-1590690486,False,39.957893,-75.173729,"[{'label': 'display', 'lat': 39.957893, 'lng':...",728,19103,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4e5ae2c1ae6099fc00db9456,UCross Game Room,Gaming Cafe,v-1590690486,False,39.956029,-75.187662,"[{'label': 'display', 'lat': 39.956029, 'lng':...",494,19104,...,3175 John F Kennedy Blvd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4ce00805aba88cfae0b04cd7,Game Creek Hunting Farms,Other Great Outdoors,v-1590690486,False,39.956915,-75.169554,"[{'label': 'display', 'lat': 39.956915, 'lng':...",1056,08098,...,1013 Kings Hwy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,52eb0112498e187a39e86852,Philly Game Forge,Coworking Space,v-1590690486,False,39.948751,-75.145671,"[{'label': 'display', 'lat': 39.94875115969555...",3190,19106,...,239 Chestnut St,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,51786d11e4b04c7998081306,The Players Station for the Biggest Video Game...,Arcade,v-1590690486,False,39.965458,-75.182457,"[{'label': 'display', 'lat': 39.96545791625976...",1061,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,50ba54d2e4b063c748fd1758,Game Pieces,Sculpture Garden,v-1590690486,False,39.954138,-75.163989,"[{'label': 'display', 'lat': 39.95413789401551...",1538,19102,...,1401 John F Kennedy Blvd,Broad Street,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,530b6f3e498e4d77c2ef3cb2,Philadelphia Game Lab,Non-Profit,v-1590690486,False,39.950372,-75.168951,"[{'label': 'display', 'lat': 39.950372, 'lng':...",1264,19103,...,1701 Walnut St,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,4b411210f964a5207bc025e3,GameStop,Video Game Store,v-1590690486,False,39.952158,-75.157176,"[{'label': 'display', 'lat': 39.95215752178449...",2148,19107,...,1001 Market St,at The Gallery 2 (Mall Level),Center City East,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,4eff533f0e01089c5525e792,The Video Game Store,Gaming Cafe,v-1590690486,False,39.948439,-75.166460,"[{'label': 'display', 'lat': 39.94843867806823...",1557,18101,...,809 Hamilton St,8th Street,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,4b454e7df964a520510a26e3,GameStop,Video Game Store,v-1590690486,False,39.941848,-75.150980,"[{'label': 'display', 'lat': 39.941848, 'lng':...",3067,19147,...,505 South St,at 5th,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# Drop exhibit, sculpture garden, art gallery (art), hunting establishments, unrelated restaurant, 
# "None" (all dupes and already accounted for), The Head Game (barber)
dataframe_game = dataframe_game[dataframe_game.categories != 'Exhibit']
dataframe_game = dataframe_game[dataframe_game.categories != 'Other Great Outdoors']
dataframe_game = dataframe_game[dataframe_game.categories != 'Scultpture Garden']
dataframe_game = dataframe_game[dataframe_game.categories != 'Art Gallery']
dataframe_game = dataframe_game[dataframe_game.categories != 'Vegetarian / Vegan Restaurant']
dataframe_game.dropna(subset=['categories'],inplace=True)
dataframe_game = dataframe_game[dataframe_game.name != 'The Head Game']
dataframe_game.head(50)

,id,name,categories,referralId,hasPerk,lat,lng,location.labeledLatLngs,location.distance,location.postalCode,...,location.address,location.crossStreet,location.neighborhood,delivery.id,delivery.url,delivery.provider.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.icon.name,venuePage.id
1,4e5ae2c1ae6099fc00db9456,UCross Game Room,Gaming Cafe,v-1590690486,False,39.956029,-75.187662,"[{'label': 'display', 'lat': 39.956029, 'lng':...",494,19104,...,3175 John F Kennedy Blvd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,52eb0112498e187a39e86852,Philly Game Forge,Coworking Space,v-1590690486,False,39.948751,-75.145671,"[{'label': 'display', 'lat': 39.94875115969555...",3190,19106,...,239 Chestnut St,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,51786d11e4b04c7998081306,The Players Station for the Biggest Video Game...,Arcade,v-1590690486,False,39.965458,-75.182457,"[{'label': 'display', 'lat': 39.96545791625976...",1061,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,50ba54d2e4b063c748fd1758,Game Pieces,Sculpture Garden,v-1590690486,False,39.954138,-75.163989,"[{'label': 'display', 'lat': 39.95413789401551...",1538,19102,...,1401 John F Kennedy Blvd,Broad Street,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,530b6f3e498e4d77c2ef3cb2,Philadelphia Game Lab,Non-Profit,v-1590690486,False,39.950372,-75.168951,"[{'label': 'display', 'lat': 39.950372, 'lng':...",1264,19103,...,1701 Walnut St,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,4b411210f964a5207bc025e3,GameStop,Video Game Store,v-1590690486,False,39.952158,-75.157176,"[{'label': 'display', 'lat': 39.95215752178449...",2148,19107,...,1001 Market St,at The Gallery 2 (Mall Level),Center City East,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,4eff533f0e01089c5525e792,The Video Game Store,Gaming Cafe,v-1590690486,False,39.948439,-75.166460,"[{'label': 'display', 'lat': 39.94843867806823...",1557,18101,...,809 Hamilton St,8th Street,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,4b454e7df964a520510a26e3,GameStop,Video Game Store,v-1590690486,False,39.941848,-75.150980,"[{'label': 'display', 'lat': 39.941848, 'lng':...",3067,19147,...,505 South St,at 5th,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,4cc7644f456e9eb0274cc33e,GameStop,Video Game Store,v-1590690486,False,39.917898,-75.186718,"[{'label': 'display', 'lat': 39.91789773792953...",4254,19145,...,2308 W Oregon Ave,at 23rd St,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,4b088c7cf964a520240e23e3,GameStop,Video Game Store,v-1590690486,False,39.922598,-75.169965,"[{'label': 'display', 'lat': 39.92259771812291...",3847,19148,...,2201 S Broad St,at Jackson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=11) # generate map centred around 30th Street Station

# add a red circle marker to represent 30th Street Station
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='30th Street Station',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the arcades as blue circle markers
for lat, lng, label in zip(dataframe.lat, dataframe.lng, dataframe.name):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label.replace('\'',''),
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)
    
# add the game related venues as green circle markers
for lat, lng, label in zip(dataframe_game.lat, dataframe_game.lng, dataframe_game.name):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='green',
        popup=label.replace('\'',''),
        fill = True,
        fill_color='green',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map